# 第二章 Singleton Design Pattern

singleton是今天你的class只會產生出一個實例，很常被用來作為全域的logging or database operation。singleton的用途可以說是下面這三點

 - 確保只有一個實體產生
 - 提供全域存取實體的方法
 - 控制平行process分享的資源

In [131]:
class Singleton:
    
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super().__new__(cls)
        
        return cls.instance

a = Singleton()
b = Singleton()

print(f'id of a {id(a)}, id of b {id(b)}')

id of a 4447030072, id of b 4447030072


上面可以看出呼叫了兩次Singleton都是回傳同一個instance! 但是呢，假如這樣的程式碼是在多執行緒下面跑的話，就會有機會造成產生兩個實體，所以我們必須做點事情讓它不會有這樣的問題產生。

In [130]:
import threading

lock = threading.Lock()

class SingletonWithLock:
    
    def __new__(cls):
        lock.acquire()
        
        if not hasattr(cls, 'instance'):
            cls.instance = super().__new__(cls)
        
        lock.release()
        
        return cls.instance
        
a = SingletonWithLock()
print(f'id of a {id(a)}')

id of a 4447008640


不過python本身其實有著比起一般語言多了很多所謂的atomic operation，也就是說再多執行緒下，由於這些operation本來就是無法分割的，因此不需要做鎖的保護，但是這前提是你的環境是用CPython(有著Global Interpreter Lock)，為什麼GIL就可以不用特別做lock處理呢? 下面是官方文檔片段[詳細可以點此](https://docs.python.org/3/faq/library.html#what-kinds-of-global-value-mutation-are-thread-safe)

> Each bytecode instruction and therefore all the C implementation code reached from each instruction is therefore atomic from the point of view of a Python program.

因此今天你想要判斷程式碼是不是thread safe in GIL，簡單說就是你只要確認你要的動作是不是只需要一個bytecode就完成！ 那麼我們要怎麼來看bytecode咧！？

幸好幸好python的內建模組**dis**，可以用來分析function, generator, method, string of source code, or a code object

In [162]:
import dis


dis.dis("a = Singleton()")


  1           0 LOAD_NAME                0 (Singleton)
              2 CALL_FUNCTION            0
              4 STORE_NAME               1 (a)
              6 LOAD_CONST               0 (None)
              8 RETURN_VALUE


這邊解析出他的bytecode用了五個，但是實際上只做了call_function，雖然你會覺得奇怪Singleton的__new__裡面做了那麼多事情，最好只要呼叫function就結束！沒錯他裡面的確做了很多事情，但是因為在他做完那些事情前，並不會把鎖交給其他人，所以其實在GIL環境時，是不需要加鎖的～～ 這個跟python的sort一樣，他也是atomic的operation。

不過你今天確實無法保證，你的code執行環境會有GIL，所以最保險的狀況的確是加鎖會比較好！

## singleton use cases

 - lazy instantiation
 
比如有時候我們使用了部分第三方套件(取決於他怎麼實作的)，他會觸發一些class的instantiation，但是我們可能只用需要用到這模組的某功能，並不需要那些class的instance。因此這些套件在實作上可以考慮lazy instantiation。

In [26]:
class SingletonLazy:
    
    __instance = None # python use name mangling to prevent acccess from outside
    
    def __init__(self):
        if not SingletonLazy.__instance:
            print("__init__ method called.")
        else:
            print(f"instance aleardy created: {self.getInstance()}")
            
    @classmethod
    def getInstance(cls):
        if not cls.__instance:
            cls.__instance = SingletonLazy()
        return cls.__instance
    
a = SingletonLazy()

SingletonLazy.getInstance()

b = SingletonLazy()

__init__ method called.
__init__ method called.
instance aleardy created: <__main__.SingletonLazy object at 0x108df27b8>


 - module level singleton
 
值得注意的是，在python中，所有的module預設都是singleton喔。
當你import module時他會做instaniate，然後如果再一次的import，他會直接回傳該instance。


## The Monostate Singleton pattern(Borg pattern)

接下來介紹一下singleton的變形，其實大家總有自己的想法，同樣的事情每人都有自己的定義!

> GoF's Singleton design pattern says that there should be one and only one object of a class.

GoF強調的是class只能有一個實體，但是呢Alex Martelli說需要單一的應該是狀態，也就是說可以多個instance但是他們分享著同樣的狀態，下面就來試著實作一下。

In [29]:
class Borg:
    _shared_state = {}
    
    def __new__(cls):
        instance = super().__new__(cls)
        instance.__dict__ = cls._shared_state
        return instance
    
a = Borg()
a.x = 2

b = Borg()
print(b.x)

print(f'x of a {a.x}, x of b {b.x}')
print(f'statue of a,b:({(a.__dict__, b.__dict__)})')

2
x of a 2, x of b 2
statue of a,b:(({'x': 2}, {'x': 2}))


上面這樣的代碼就輕易地實現了共用屬性。

這篇章節講到了singleton是屬於creational pattern的一環，剛剛好！python有個metaclass，跟class, object的創建很有關係！！ 就是**metaclass**，想要看詳細的說明可以[看我的部落格心得](http://sillygod.github.io/2015/01/25/understanding-python-metaclass/)，或是想要看定義的話就到[PEP3115](https://www.python.org/dev/peps/pep-3115/#invoking-the-metaclass)。

裡面提到其中一段

>In the current metaclass system, the metaclass object can be any callable type.

所以其實，今天你metaclass帶入的只要是能夠被呼叫的！ 因此最簡單的方式你可以使用function，或者你堅持使用class也是可以啦以下示範

In [52]:
class Snow:
    
    def __init__(self):
        self.name = "snow"

def func_meta(name, bases, attributes):
    # 從PEP的定義你會看到metaclass他會傳三個參數給你
    print(f'name: {name}, bases: {bases}, attributes: {attributes}\n')
    return Snow

def func_meta_bad():
    # 換成這個會爆掉喔
    print('will enter?')
    
    
class TestMeta(metaclass=func_meta):
    
    def __init__(self):
        pass
    
m = TestMeta()
print(f"ok Let's see what TestMeta become {m.name}")


name: TestMeta, bases: (), attributes: {'__module__': '__main__', '__qualname__': 'TestMeta', '__init__': <function TestMeta.__init__ at 0x108d68d08>}

ok Let's see what TestMeta become snow


從上面簡單的範例，可以看出他改變了class產生出來的物件，因為我在裡面做了換取的動作！

那麼接著使用class的方式來示範一次，以下做個示範

In [126]:
def print_decorator(clsname, func):
    # 這個用處只是故意方便來以下demo，metaclass的運作流程
    def print_it_before_call(*args, **kwargs):
        print(f'{clsname} calling {func.__name__}')
        result = func(*args, **kwargs)
        return result
    
    return print_it_before_call

class MetaSingleton(type):
    
    __instances = {}
    
    def __new__(cls, name, bases, attrs):
        print('enter meta __new__')
        # lookup class attrs and set callable attr to
        # prepend print
        for key, value in attrs.items():
            if hasattr(value, '__call__'):
                # do print decorator
                attrs[key] = print_decorator(name, value)
        
        # interesting problem: https://stackoverflow.com/questions/41343263/provide-classcell-example-for-python-3-6-metaclass
        return type.__new__(cls, name, bases, attrs)
    
    def __call__(cls, *args, **kwargs):
        print('enter meta __call__')
        if cls not in cls.__instances:
            cls.__instances[cls] = super().__call__(*args, **kwargs) 
            # 這邊實際上等價於 cls.__instances[cls] = type(name, bases, attrs)(*args, **kwargs)
        
        return cls.__instances[cls]
    

class TestMetaSingleton(metaclass=MetaSingleton):
    
    def __init__(self):
        pass

    def __new__(cls):
        return super().__new__(cls)
    
    def bark(self):
        print('hello')
    
a = TestMetaSingleton()
b = TestMetaSingleton()

print(f'id of (a, b) {(id(a), id(b))}')

enter meta __new__
enter meta __call__
TestMetaSingleton calling __new__
TestMetaSingleton calling __init__
enter meta __call__
id of (a, b) (4446069872, 4446069872)


從上面的結果，應該可以看出呼叫順序，不意外的都是meta class先，可以看出來`__call__`這邊促使了TestMetaSingleton的`__new__`, `__init__` 先後被呼叫，但是第二次就沒有被再次呼叫的原因，因為這裡我們做了singleton。

其實python的metaclass可能有點奇淫技巧了拉，這邊我只是出於興趣，所以特別實作`print_decorator`，解析metaclass一下。有興趣的，在自行去鑽研吧。